# Telecom X – Desafio de Análise de Evasão de Clientes

A **Telecom X**, uma das principais empresas de telecomunicações do mercado, vem enfrentando um **alto índice de cancelamentos de clientes (churn)**. Esse fenômeno impacta diretamente a receita e a competitividade da empresa, tornando urgente a identificação dos **fatores que levam os clientes a encerrar seus contratos**.

Como **assistente de análise de dados**, você integrará o projeto **"Churn de Clientes"**, que tem como objetivo compreender profundamente o comportamento da base de clientes e gerar **insights estratégicos** que apoiem a criação de ações para **reduzir a evasão**.

Neste desafio, você irá trabalhar com um **dataset realista** disponibilizado via API no formato JSON, contendo informações demográficas, tipos de serviços contratados e histórico de relacionamento com a empresa.

---

## Etapas do Desafio

1. **Extração (E – Extract)**  
   - Carregar os dados diretamente da API da Telecom X usando Python.  
   - Garantir que a leitura seja feita no formato adequado e salvar uma cópia local para reprodutibilidade.

2. **Transformação (T – Transform)**  
   - Limpar inconsistências, padronizar formatos, tratar valores ausentes e ajustar tipos de dados.  
   - Preparar o dataset para análises exploratórias e modelagem.

3. **Carga (L – Load)**  
   - Armazenar os dados tratados em um formato adequado (ex.: Parquet ou CSV) para uso posterior pela equipe de Data Science.

4. **Análise Exploratória (EDA)**  
   - Criar visualizações estratégicas para identificar padrões, correlações e tendências relacionadas ao churn.  
   - Gerar um relatório com insights claros e objetivos que servirão de base para o desenvolvimento de modelos preditivos.

---

## O que você vai praticar
✅ Importar e manipular dados a partir de uma API de forma eficiente.  
✅ Aplicar conceitos práticos de **ETL** na preparação de dados para análise.  
✅ Criar visualizações e métricas para **identificar padrões e tendências**.  
✅ Desenvolver uma análise exploratória de dados (EDA) para apoiar **decisões estratégicas**.  

---

Agora é sua vez! 🚀  
Use seus conhecimentos para transformar **dados brutos** em **informações estratégicas** e contribuir para que a **Telecom X** retenha mais clientes e aumente sua vantagem competitiva.


## O que é ETL e por que isso é importante

**ETL** significa **Extração (Extract), Transformação (Transform) e Carga (Load)**.  
É o processo fundamental que leva dados de fontes diversas (APIs, arquivos, bancos), **padroniza e limpa** essas informações, e as **carrega** para um destino confiável (ex.: data warehouse, camadas “processed” ou “gold”).

### Por que ETL é essencial em Data Science e Analytics?
- **Qualidade e confiabilidade**: remove inconsistências, trata nulos e garante tipos corretos.
- **Reprodutibilidade**: documenta o fluxo de dados de ponta a ponta.
- **Performance analítica**: prepara dados prontos para EDA, modelagem e dashboards.
- **Governança**: cria trilhas de auditoria (de onde veio e como foi transformado).

No fim, **um bom ETL acelera a análise** e dá segurança para que as decisões e modelos preditivos sejam baseados em dados sólidos.


## ETL — Etapa E (Extração dos Dados)

**Objetivo desta seção:** carregar os dados da API (JSON) da Telecom X a partir do link *raw* do GitHub, **validar a resposta** e **criar uma cópia local** dos dados brutos para reprodutibilidade.

**Entradas:**
- URL (raw) do JSON no GitHub

**Saídas:**
- `df_raw` (DataFrame com os dados brutos)
- Arquivo salvo em `data/raw/TelecomX_Data.json` (cópia local)


In [9]:
# Imports e configurações
import os
from pathlib import Path
import io, json
import pandas as pd
import numpy as np

# Opcional: útil para checar status HTTP
try:
    import requests
    HAS_REQUESTS = True
except ImportError:
    HAS_REQUESTS = False

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Cria estrutura para dados brutos (se ainda não existir)
RAW_DIR = Path("data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)


### Fonte de dados (RAW)

- Link **RAW** do JSON (GitHub):
  - `https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json`

**Plano de leitura:**
1. (Opcional) Verificar se a URL responde com status 200.
2. Ler o JSON em um `DataFrame` (`pandas.read_json`).
3. Salvar uma **cópia local** em `data/raw/TelecomX_Data.json` para reprodutibilidade.
4. Exibir **amostra**, **shape** e **tipos**.

In [11]:
# URL do arquivo JSON bruto (API)
DATA_URL_RAW = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"
LOCAL_RAW_PATH = RAW_DIR / "TelecomX_Data.json"

# 1) Baixar diretamente da API (link raw) usando Python
if not HAS_REQUESTS:
    raise ImportError("O pacote 'requests' é necessário para a etapa de extração direta da API.")

resp = requests.get(DATA_URL_RAW, timeout=30)
print("Status HTTP:", resp.status_code)
resp.raise_for_status()  # garante 2xx

# 2) Converter o payload recebido diretamente para DataFrame do pandas
#    (garante que é via Python -> API -> DataFrame)
try:
    # se o JSON é uma lista de registros, isso funciona bem:
    df_raw = pd.read_json(io.StringIO(resp.text))
except ValueError:
    # fallback: carregar via json.loads -> DataFrame
    data = json.loads(resp.text)
    df_raw = pd.DataFrame(data)

# ✅ Garantias explícitas dos requisitos
assert isinstance(df_raw, pd.DataFrame), "Falha: os dados não foram convertidos em um DataFrame."
assert not df_raw.empty, "Atenção: DataFrame vazio após leitura da API."

# 3) Salvar cópia local (reprodutibilidade)
LOCAL_RAW_PATH.write_text(resp.text, encoding="utf-8")
print(f"Arquivo bruto salvo em: {LOCAL_RAW_PATH.resolve()}")

# 4) Visão rápida (sem despejar tudo na tela)
print("Tipo de df_raw:", type(df_raw), "| nome:", type(df_raw).__name__)
print("Shape:", df_raw.shape)
display(df_raw)
display(df_raw.info())


Status HTTP: 200
Arquivo bruto salvo em: D:\Users\rodri\Documents\ONE + Alura\ONE G8\Challenges\challenge-telecomX\data\raw\TelecomX_Data.json
Tipo de df_raw: <class 'pandas.core.frame.DataFrame'> | nome: DataFrame
Shape: (7267, 6)


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
...,...,...,...,...,...,...
7262,9987-LUTYD,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
7263,9992-RRAMN,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7264,9992-UJOEL,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7265,9993-LHIEB,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Two year', 'PaperlessBilling': '..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


None

# Transformação

# Carga e análise

#📄Relatorio Final